<a href="https://colab.research.google.com/github/myazann/Text-Generation/blob/main/Data_Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install git-python==1.0.3

import os
import csv
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gc
from dataclasses import dataclass, field
from typing import Optional
from getpass import getpass

Getting the data into a dataframe.



In [ ]:
os.chdir("/content/drive/My Drive")

!cp ChatExport_2021-01-01.zip /content

os.chdir("/content")

!unzip ChatExport_2021-01-01.zip
!rm ChatExport_2021-01-01.zip

In [ ]:
f = open('ChatExport_2021-01-01/result.json',encoding="utf8") 
messages = json.load(f)

orig_df = pd.DataFrame(columns = ["Writer","Messages","Date"])
i = 0

for message in messages["messages"]:
    if message["type"] == "message":
        orig_df = orig_df.append({
            "Writer":message["from"],
            "Messages":message["text"],
            "Date":message["date"]
                  }, ignore_index = True)
        if i%100 == 0:
            print(i)
        i += 1


Preprocessing

In [ ]:
data_df = orig_df

data_df["Writer"] = np.where(data_df["Writer"].isna(), "Aykut", data_df["Writer"])

data_df["Messages_Lower"] = data_df["Messages"].str.lower()
data_df["Messages_Lower"] = data_df["Messages_Lower"].str.strip()
data_df["Word_Splits"] = data_df["Messages_Lower"].str.split()

data_df = data_df.dropna()

data_df["Messages"] = data_df["Messages_Lower"]

data_df = data_df.drop("Messages_Lower", axis = 1)

data_df = data_df.loc[data_df["Messages"] != ""]

writers = ["Mert Yazan", "Kumru", "Atamert", "Abidin Dekar", "Aykut"]

data_df = data_df.loc[data_df["Writer"].isin(writers)]

i = 0
data_lengths = []
msg_words = data_df["Word_Splits"].values

while i < len(msg_words):
    data_lengths.append(len(msg_words[i]))        
    i += 1 
    
data_df["Lengths"] = data_lengths    

data_df = data_df.loc[data_df["Lengths"] >= 1]

data_df["Messages"] = np.where((data_df["Messages"].str.contains("hahaha")) & (data_df["Lengths"] == 1),"hahaha",data_df["Messages"])

data_df["Writer"] = np.where(data_df["Writer"] == "Mert Yazan", "Mert", data_df["Writer"])
data_df["Writer"] = np.where(data_df["Writer"] == "Abidin Dekar", "Abidin", data_df["Writer"])

data_df["input"] = data_df["Writer"] + " " + data_df["Messages"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Creating the labels as the next sentences, and splitting them into train and val sets.

In [ ]:
label_len = len(data_df["input"].values)-1

data = []
labels = []
text = ""

for i in range(label_len):
  labels.append(data_df["input"].values[i+1])
  for j in range (1):
    text = text + " " + data_df["input"].values[i+j] 
  data.append(text)
  text = ""

labels = np.asarray(labels)
data = np.asarray(data)

df2 = pd.DataFrame()

df2["Written_Text"] = data
df2["Answer_Text"] = labels

train, val = train_test_split(df2, test_size = 0.2)

train.to_csv("telegram_chatbot_train.csv", index = False, encoding = "utf-8")

val.to_csv("telegram_chatbot_val.csv", index = False, encoding = "utf-8")